## Potential Modifications:
**other HuggingFace backbones  
other heads designs  
other architecture tricks like using hidden layer activations  
other pre and post processing  
question and answer instead of NER  
other learning schedules, optimizers, etc  
sliding window training  
adjust attention of LongFormer  
more folds  
data augmentation  
external data  **

### TURN OFF INTERNET
In code competitions, we must turn internet off to make a submission. This starter notebook shows how to download any transformer of your choice to a Kaggle dataset

# Import

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os

from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
import pdb
import torch
from torch import cuda

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch


# Configuration

In [ ]:
# import os
# # DECLARE HOW MANY GPUS YOU WISH TO USE. 
# # KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
# os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# # VERSION FOR SAVING MODEL WEIGHTS
# VER=26

# # IF VARIABLE IS NONE, THEN NOTEBOOK COMPUTES TOKENS
# # OTHERWISE NOTEBOOK LOADS TOKENS FROM PATH
# LOAD_TOKENS_FROM = '../input/py-bigbird-v26'

# # IF VARIABLE IS NONE, THEN NOTEBOOK TRAINS A NEW MODEL
# # OTHERWISE IT LOADS YOUR PREVIOUSLY TRAINED MODEL
# LOAD_MODEL_FROM = '../input/py-bigbird-v26'

# # IF FOLLOWING IS NONE, THEN NOTEBOOK 
# # USES INTERNET AND DOWNLOADS HUGGINGFACE 
# # CONFIG, TOKENIZER, AND MODEL
# DOWNLOADED_MODEL_PATH = '../input/py-bigbird-v26' 

# if DOWNLOADED_MODEL_PATH is None:
#     DOWNLOADED_MODEL_PATH = 'model'    
# MODEL_NAME = 'google/bigbird-roberta-base'

In [ ]:
model_checkpoints = "allenai/longformer-base-4096"

In [ ]:
train_df = pd.read_csv('/kaggle/input/feedback-prize-2021/train.csv')

In [ ]:
test_names, train_texts = [], []
for f in list(os.listdir('../input/feedback-prize-2021/train')):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
    
doc_df = pd.DataFrame({'id': test_names, 'text': train_texts})
doc_df.head()

# Create Dataset: Convert Train Text to NER Labels

In [ ]:
entities = []
for index,row in doc_df.iterrows():
    length_text = row['text'].split().__len__()
    ent = ["O" for i in range(length_text)]

    for idx,r in train_df[train_df['id'] == row['id']].iterrows():
        
        pred_idx = r['predictionstring'].split()
        ent[int(pred_idx[0])] = f"B-{r['discourse_type']}"

        for i in pred_idx[1:]:
            ent[int(i)] = f"I-{r['discourse_type']}"
        

    entities.append(ent)



In [ ]:
doc_df['elements'] = entities
doc_df.head()

In [ ]:
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

doc_df['labels'] = doc_df['elements'].apply(lambda x: [labels_to_ids[i] for i in x])


In [ ]:
doc_df.head()

### Check MAX LENGTH (we use 1024 wide tokens since the majority of text is less than 1024 tokens.)

In [ ]:
### Check MAX LENGTH (we use 1024 wide tokens since the majority of text is less than 1024 tokens.)
# def calc_len(text):
#     ids = tokenizer(text)['input_ids']
#     return len(ids)

# tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-large')
# doc_df['length'] = doc_df['text'].apply(calc_len)
# do_df.describe()

In [ ]:
#Experiement with first top 3 training data
tr_df = doc_df.iloc[:3]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints,add_prefix_space=True)

# DataLoader
### outputs: tokens and attention. 
During training it also provides labels.
During inference it also provides word ids to help convert token predictions into word predictions.

In [ ]:
# IGNORE_INDEX = -100
# NON_LABEL = -1
# DO WE LABEL ALL SUBTOKENS???

LABEL_ALL_SUBTOKENS = True

class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len, get_wids):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.get_wids = get_wids # for validation

  def __getitem__(self, index):
        # GET TEXT AND WORD LABELS 
        text = self.data.text[index]        
        word_labels = self.data.elements[index] if not self.get_wids else None

        # TOKENIZE TEXT
        encoding = self.tokenizer(text.split(),
                             is_split_into_words=True,
                             #return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        word_ids = encoding.word_ids()  
        
        # CREATE TARGETS
        if not self.get_wids:
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:                            
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:              
                    label_ids.append( labels_to_ids[word_labels[word_idx]] )
                else:
                    if LABEL_ALL_SUBTOKENS:
                        label_ids.append( labels_to_ids[word_labels[word_idx]] )
                    else:
                        label_ids.append(-100)
                previous_word_idx = word_idx
            encoding['labels'] = label_ids

        # CONVERT TO TORCH TENSORS
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        if self.get_wids: 
            word_ids2 = [w if w is not None else -1 for w in word_ids]
            item['wids'] = torch.as_tensor(word_ids2)
        
        return item

  def __len__(self):
        return self.len

In [ ]:
# CHOOSE VALIDATION INDEXES (that match my TF notebook)
IDS = tr_df.id.unique()
print('There are',len(IDS),'train texts. We will split 90% 10% for validation.')

# TRAIN VALID SPLIT 90% 10%
np.random.seed(42)
train_idx = np.random.choice(np.arange(len(IDS)),int(0.9*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)
np.random.seed(None)

In [ ]:
# CREATE TRAIN SUBSET AND VALID SUBSET
data = tr_df[['id','text', 'elements']]
train_dataset = data.loc[data['id'].isin(IDS[train_idx]),['text', 'elements']].reset_index(drop=True)
test_dataset = data.loc[data['id'].isin(IDS[valid_idx])].reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, 1024, False)
testing_set = dataset(test_dataset, tokenizer, 1024, True)

In [ ]:
# TRAIN DATASET AND VALID DATASET
train_params = {'batch_size': 3,
                'shuffle': True,
                'num_workers': 2,
                'pin_memory':True
                }

test_params = {'batch_size': 3,
                'shuffle': False,
                'num_workers': 2,
                'pin_memory':True
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# TEST DATASET
# test_texts_set = dataset(test_texts, tokenizer, config['max_length'], True)
# test_texts_loader = DataLoader(test_texts_set, **test_params)